## --------------------------------------------------------------------------------------------------------------------------------------
# KELOMPOK 5 A2
## --------------------------------------------------------------------------------------------------------------------------------------
## ANGGOTA KELOMPOK :
### Natalie Fuad                       : 2109106040 (KETUA)
### Agustina Dwi Maharani     : 2109106037
### Sherina Laraswati              : 21091050
## --------------------------------------------------------------------------------------------------------------------------------------

In [31]:
import splitfolders

In [32]:
splitfolders.ratio('flowers/', output="dataset-output", seed=42, ratio=(0.7,0.25,0.05), group_prefix=None)

Copying files: 4894 files [00:06, 743.37 files/s]


## 1. IMPORT LIBRARY

In [33]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpig
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator